# MMAD Dataset – Full Structure & EDA

이 노트북은 **MMAD 데이터셋의 전체 구조를 이해**하기 위한 목적의 EDA입니다.

### 이 노트북에서 다루는 것
1. `mmad.json` **전체 스키마 구조 정리**
2. 데이터셋 **폴더 트리 자동 탐색**
3. 질문 타입 / 옵션 / shot(template) 구조 통계
4. mask 존재 여부 및 활용 가능성
5. 실험 설계를 위해 반드시 필요한 **핵심 인사이트 요약**


In [ ]:
from pathlib import Path
import os, json
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from PIL import Image
import matplotlib.pyplot as plt


## 0. Dataset 경로 설정

In [ ]:
MMAD_ROOT = Path(os.environ.get("MMAD_ROOT", "/Volumes/T7/Dataset/MMAD")).expanduser()
MMAD_JSON = MMAD_ROOT / "mmad.json"
print("MMAD_ROOT:", MMAD_ROOT)
print("mmad.json exists:", MMAD_JSON.exists())


## 1. mmad.json 로드 및 Top-level 구조

In [ ]:
with MMAD_JSON.open("r", encoding="utf-8") as f:
    mmad = json.load(f)

print("Total images:", len(mmad))
sample_key = next(iter(mmad.keys()))
print("Sample key:", sample_key)
print("Fields:", mmad[sample_key].keys())


### 1.1 mmad.json 기본 스키마 설명

- `image_path`
- `conversation` (객관식 MCQ)
- `mask_path`
- `similar_templates`
- `random_templates`


## 2. Conversation 구조 분석

In [ ]:
def parse_conversation_stats(mmad):
    rows = []
    for k, v in mmad.items():
        for q in v.get("conversation", []):
            rows.append({
                "image": k,
                "type": q.get("type"),
                "num_options": len(q.get("Options", {})),
                "has_annotation": q.get("annotation", False)
            })
    return pd.DataFrame(rows)

conv_df = parse_conversation_stats(mmad)
conv_df["type"].value_counts()


## 3. Shot(Template) 구조

In [ ]:
tmpl_df = pd.DataFrame([
    {
        "num_similar": len(v.get("similar_templates", [])),
        "num_random": len(v.get("random_templates", []))
    }
    for v in mmad.values()
])
tmpl_df.describe()


## 4. Mask 구조

In [ ]:
mask_df = pd.DataFrame([
    {"has_mask": bool(v.get("mask_path"))}
    for v in mmad.values()
])
mask_df["has_mask"].value_counts()


## 5. 폴더 트리 (상위)

In [ ]:
def print_tree(path: Path, depth=2, prefix=""):
    if depth < 0: return
    for p in sorted(path.iterdir()):
        print(prefix + ("📁 " if p.is_dir() else "📄 ") + p.name)
        if p.is_dir():
            print_tree(p, depth-1, prefix+"  ")

print_tree(MMAD_ROOT, depth=3)


## 6. 핵심 요약

- MMAD는 **객관식 멀티모달 추론 평가용 데이터셋**
- 평가 지표는 **MCQ Accuracy**
- Shot은 학습이 아니라 **추론 시 정상 reference 제공**
- mask는 보조 정보
